In [23]:
#This is the edge detection starter code

import cv2
from PIL import ImageGrab
import time 
import numpy as np
from directkeys import PressKey, ReleaseKey, W, A, S, D
import pyautogui

def roi(img, vertices):
    mask=np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked=np.bitwise_and(mask, img)
    return masked

def process_image(original_image):
    processed_image=cv2.Canny(original_image, threshold1=130, threshold2=131)
    vertices=np.array([[0, 600], [0, 325], [260, 300], [540, 300], [800, 325], [800, 600]], np.int32)
    processed_image=roi(processed_image, [vertices])
    
    return processed_image

while True:
    screen=np.array(ImageGrab.grab(bbox=(125,125,925,725)))
    cv2.imshow('Screen', process_image(screen))
    
    if(cv2.waitKey(30)==27):
        break
cv2.destroyAllWindows()

In [17]:
from grabscreen import grab_screen
import cv2
import pyautogui

while True:
#     screen=grab_screen(region=(125, 125, 925, 725))
    screen=grab_screen(region=(125, 125+250, 925, 725))
    screen=cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
    cv2.imshow('video', screen)
    if cv2.waitKey(30)==27:
        break
cv2.destroyAllWindows()

In [77]:
# To Create the data (recording frames and labelling with keystroke)

import pickle
import numpy as np
import cv2
import os
from getkeys import key_check
from grabscreen import grab_screen
import time

def key_to_output(keys):
    output=[0,0,0]
    
    if "A" in keys:
        output[0]=1
    elif "D" in keys:
        output[2]=1
    else:
        output[1]=1
    return output

filename="training_data.npy"

if os.path.isfile(filename):
    train=list(np.load(filename, allow_pickle=True))
    print("File Found, loading data!")
else:
    train=[]
    print("Starting Afresh!")


def main():
    
    for t in list(range(5))[::-1]:
        print(t)
        time.sleep(1)
    
    # last_time=time.time()
    while True:
        screen=grab_screen(region=(125, 125+250, 925, 725))
        screen=cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
        screen=cv2.resize(screen, (160, 70))
        keys=key_check()
        output=key_to_output(keys)
        train.append([screen, output])
    #     print(time.time()-last_time)
    #     last_time=time.time()
        if((len(train)%500)==0):
            print(len(train))
            np.save(filename, train)

main()            

Starting Afresh!
4
3
2
1
0
500
1000
1500


KeyboardInterrupt: 

In [21]:
l=np.load(filename, allow_pickle=True)
cv2.imshow('img', l[250][0])
cv2.waitKey()


-1

In [79]:
# To seperate the data in diffrent categories

from random import shuffle
import pickle
import numpy as np
import cv2
filename="training_data.npy"
train=np.load(filename, allow_pickle=True)
shuffle(train)
# count1 = count2 = count3 = 0
for f in train:
    if f[1]==[1,0,0]:
        count1+=1
        cv2.imwrite("ROI enabled\[1, 0, 0]\Left_%d.jpg"%count1, f[0])
    elif f[1]==[0,1,0]:
        count2+=1
        cv2.imwrite("ROI enabled\[0, 1, 0]\Forward_%d.jpg"%count2, f[0])
    elif f[1]==[0,0,1]:
        count3+=1
        cv2.imwrite("ROI enabled\[0, 0, 1]\Right_%d.jpg"%count3, f[0])
# print(count1, count2, count3)

In [78]:
print(count1, count2, count3)

1535 25794 1171


In [150]:
%reload_ext tensorboard

In [80]:
from keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator()

train=datagen.flow_from_directory('ROI enabled', target_size=(160, 70), batch_size=32, class_mode='categorical', shuffle=True)
# test=datagen.flow_from_directory('Aditya/test data aditya', target_size=(80, 60), batch_size=32, class_mode='categorical', shuffle=True)

Found 3633 images belonging to 3 classes.


In [81]:
#Downloading VGG16

from tensorflow.keras.applications.vgg16 import VGG16
vgg=VGG16(include_top=False, weights='imagenet', input_shape=(160,70,3))

In [82]:
# Defining the model
import tensorflow
# from keras import Model
# from keras.layers import Dense, Flatten

for layer in vgg.layers:
    layer.trainable=False

y=tensorflow.keras.layers.Flatten()(vgg.output)
x=tensorflow.keras.layers.Dense(3, activation='softmax')(y)
model=tensorflow.keras.models.Model(inputs=vgg.input, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
#Aditya

from keras.callbacks.callbacks import ModelCheckpoint
filepath="weights.best.Aditya.416.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [15]:
#create callback list

# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime,os
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
# NAME = "Self_driving_car{}".format(int(time.time()))
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir)


# checkpoint
filepath="weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [38]:
from keras import backend as K
K.get_value(model.optimizer.lr)
# K.set_value(model.optimizer.lr, 0.0001)
# K.get_value(model.optimizer.lr)

0.001

In [83]:
# from tensorflow.keras.models import load_model
# model.fit_generator(train, validation_data=test, validation_steps=len(test), epochs=2, steps_per_epoch=len(train), callbacks=callbacks_list)
model.fit_generator(train, epochs=3, steps_per_epoch=len(train), callbacks=callbacks_list)

  ...
    to  
  ['...']
Train for 114 steps
Epoch 1/3
114/114 [==============================] - 596s 5s/step - loss: 1.8945 - accuracy: 0.8057


C:\Users\LENOVO\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Epoch 2/3
114/114 [==============================] - 587s 5s/step - loss: 0.6606 - accuracy: 0.9056
Epoch 3/3
114/114 [==============================] - 389s 3s/step - loss: 0.3353 - accuracy: 0.9463


In [58]:
from tensorflow.keras.models import load_model
model1=load_model('weights.best.Aditya.hdf5')

In [85]:
import cv2
from directkeys import PressKey, ReleaseKey, W, A, S, D
from grabscreen import grab_screen
import numpy as np
import time
from getkeys import key_check


def left():
    PressKey(A)
    PressKey(W)
    ReleaseKey(D)
    
def right():
    PressKey(D)
    PressKey(W)
    ReleaseKey(A)
    
def forward():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    
def drive():
    while True:
        screen=grab_screen(region=(125, 125+250, 925, 725))
        screen=cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
        screen=cv2.resize(screen, (70, 160))
        screen4D=np.expand_dims(screen, axis=0)
        pred=model.predict(screen4D)
        print(pred)
        
        if pred[0][0]>0.5:
            right()
        
        elif pred[0][1]>0.5:
            forward()
        
        elif pred[0][2]>0.5:
            left()
            
#         elif pred[0][0]>pred[0][1] and pred[0][0]>pred[0][2]:
#             right()
            
#         elif pred[0][1]>pred[0][0] and pred[0][1]>pred[0][2]:
#             forward()
            
#         elif pred[0][2]>pred[0][0] and pred[0][2]>pred[0][1]:
#             right()
            
            
        if "T" in key_check():
            ReleaseKey(A)
            ReleaseKey(W)
            ReleaseKey(D)
            cv2.destroyAllWindows()
            break

# for t in list(range(5))[::-1]:
#     print(t)
#     time.sleep(1)
    
while True:
    if "I" in key_check():
        drive()
    if "Q" in key_check():
        break

[[7.4772182e-13 1.1119393e-08 1.0000000e+00]]
[[1.0325836e-12 1.3577097e-08 1.0000000e+00]]
[[7.0388316e-16 3.8360048e-10 1.0000000e+00]]
[[2.8205112e-16 8.5153959e-16 1.0000000e+00]]
[[2.0244006e-13 3.0343839e-30 1.0000000e+00]]
[[2.9394208e-08 1.1618665e-31 1.0000000e+00]]
[[1.5573366e-09 4.2077184e-22 1.0000000e+00]]
[[9.8675480e-03 3.4551561e-12 9.9013245e-01]]
[[6.4522820e-03 2.4155352e-05 9.9352348e-01]]
[[1.1584221e-04 2.3933264e-09 9.9988413e-01]]
[[9.8138416e-01 4.8308166e-08 1.8615834e-02]]
[[1.2968037e-08 5.9099892e-11 1.0000000e+00]]
[[9.5074620e-07 9.9613416e-01 3.8648797e-03]]
[[5.5716350e-06 3.7234153e-05 9.9995720e-01]]
[[0.11388114 0.00782892 0.8782899 ]]
[[6.5726172e-14 1.7057034e-15 1.0000000e+00]]
[[8.4976136e-08 8.8878447e-15 9.9999988e-01]]
[[1.0076226e-01 2.7037220e-04 8.9896739e-01]]
[[9.9999976e-01 2.7262416e-07 2.8638539e-10]]
[[6.3802553e-12 1.0000000e+00 5.1729728e-08]]
[[9.9998224e-01 8.8362840e-06 8.9388404e-06]]
[[2.4424531e-04 4.1558064e-06 9.9975163e-01

[[0.16887055 0.8301809  0.00094859]]
[[0.5367191  0.41187862 0.05140238]]
[[0.57642925 0.37915945 0.04441133]]
[[9.7993338e-01 1.7573767e-06 2.0064883e-02]]
[[9.9920696e-01 7.9309434e-04 5.4051069e-10]]
[[1.0000000e+00 2.0902496e-08 4.1485861e-15]]
[[2.5705171e-07 9.9999976e-01 9.6213411e-09]]
[[1.1331525e-05 8.5037820e-02 9.1495091e-01]]
[[8.1116989e-23 2.9865532e-38 1.0000000e+00]]
[[1.000000e+00 4.436089e-22 3.127445e-16]]
[[2.5616945e-03 3.2166636e-23 9.9743831e-01]]
[[9.9953246e-01 9.9628485e-17 4.6756910e-04]]
[[9.9998188e-01 1.5664258e-30 1.8123126e-05]]
[[6.847859e-13 2.093152e-21 1.000000e+00]]
[[1.7224518e-19 1.3576633e-19 1.0000000e+00]]
[[1.3310804e-09 1.4946509e-32 1.0000000e+00]]
[[0.94151634 0.05749112 0.00099268]]
[[0.9482829  0.05065851 0.00105866]]
[[0.95647585 0.02429835 0.01922571]]
[[4.5041763e-03 9.9466789e-01 8.2802027e-04]]
[[2.1266837e-06 5.6078616e-06 9.9999225e-01]]
[[3.5040264e-04 2.9755688e-06 9.9964666e-01]]
[[7.3630184e-07 7.4670101e-09 9.9999928e-01]]
[[

[[1.0000000e+00 6.1239125e-10 1.2608333e-16]]
[[1.000000e+00 7.314375e-11 1.094713e-15]]
[[9.6063184e-15 3.5047668e-04 9.9964952e-01]]
[[3.659170e-15 1.957107e-04 9.998043e-01]]
[[6.9772464e-29 6.4169998e-09 1.0000000e+00]]
[[8.313375e-17 1.437553e-10 1.000000e+00]]
[[6.7521705e-06 3.4153826e-11 9.9999321e-01]]
[[2.5786315e-03 7.1041704e-08 9.9742126e-01]]
[[4.782333e-01 5.217668e-01 9.058658e-15]]
[[9.9999952e-01 4.8889325e-07 8.6119698e-09]]
[[1.4380364e-01 8.5619640e-01 4.6677069e-14]]
[[7.6605588e-01 2.3394415e-01 9.9251887e-09]]
[[1.9072128e-03 1.2245922e-16 9.9809283e-01]]
[[1.1867231e-05 9.9998808e-01 3.9064962e-13]]
[[1.0451155e-03 9.9895489e-01 3.9939461e-09]]
[[0.92806125 0.06952965 0.00240914]]
[[9.9963963e-01 3.2204384e-04 3.8323000e-05]]
[[0.00108935 0.99788046 0.00103021]]
[[7.7954549e-27 1.0000000e+00 1.4527394e-08]]
[[3.1424513e-28 9.9970019e-01 2.9981718e-04]]
[[1.0418458e-19 5.3463166e-04 9.9946541e-01]]
[[7.7021295e-14 1.2899846e-09 1.0000000e+00]]
[[2.4899731e-25 9.

[[2.5175090e-10 2.2318798e-13 1.0000000e+00]]
[[9.7886805e-06 1.7257791e-04 9.9981767e-01]]
[[1.0906359e-01 5.2570822e-06 8.9093119e-01]]
[[9.9999297e-01 8.7444960e-07 6.1810119e-06]]
[[2.8647904e-04 9.9127523e-05 9.9961436e-01]]
[[3.1296577e-13 1.0000000e+00 1.9591127e-14]]
[[0.02044471 0.8902422  0.08931313]]
[[5.856500e-11 2.611680e-02 9.738832e-01]]
[[1.5496380e-09 9.9997866e-01 2.1357535e-05]]
[[2.1133917e-07 9.9997044e-01 2.9365925e-05]]
[[2.4569209e-07 9.9999046e-01 9.2399123e-06]]
[[1.0000000e+00 1.3567958e-08 4.5169934e-08]]
[[1.000000e+00 1.415108e-08 4.131844e-08]]
[[9.9999857e-01 4.4090007e-07 9.5227335e-07]]
[[9.9999833e-01 1.9652696e-10 1.6582503e-06]]
[[8.2037623e-05 9.5246327e-01 4.7454618e-02]]
[[1.4440515e-11 6.6563310e-08 9.9999988e-01]]
[[5.5863775e-02 6.7903355e-05 9.4406837e-01]]
[[0.02775543 0.03147094 0.9407737 ]]
[[9.9645472e-01 2.5014672e-06 3.5428193e-03]]
[[0.00629047 0.16362509 0.83008444]]
[[8.1298754e-02 5.4134111e-05 9.1864717e-01]]
[[9.9899465e-01 9.716

In [90]:
from keras.models import load_model
model1=load_model('weights.best.hdf5')

In [24]:
model.save('Training_Acc_70_red_bonut.h5')